In [5]:
import tensorflow as tf
#from tensorflow import keras
from tensorflow.keras.applications import vgg16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense,Dropout

vgg16_model = vgg16.VGG16(weights=None,classes=2,input_shape=(256,256,3))
# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolutionfor layer in vgg16_model.layers[:-1]:
for layer in vgg16_model.layers[:-1]:
    classifier.add(layer)
for layer in classifier.layers:
    layer.trainable = False
classifier.add(Dense(1, activation='relu'))


opt = tf.keras.optimizers.Adam(learning_rate=0.02)
classifier.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])

#classifier.compile(optimizer = 'adam', loss='binary_crossentropy', metrics=['accuracy'])
classifier.summary()

    

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 64, 64, 256)      

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
train_datagen = ImageDataGenerator(rescale = 1./255,
shear_range = 0.2,
zoom_range = 0.2,
horizontal_flip = True)
# print(train_datagen)
test_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory(r"C:\Users\admin\Desktop\data\train",
target_size = (256,256),
batch_size = 32,
class_mode = 'binary')
# print(test_datagen)
test_set = test_datagen.flow_from_directory(r"C:\Users\admin\Desktop\data\validation",
target_size = (256,256),
batch_size = 32,
class_mode = 'binary')

Found 1368 images belonging to 2 classes.
Found 900 images belonging to 2 classes.


In [7]:
history = classifier.fit(training_set, 
 
                    validation_data= test_set, 
                    
                    epochs=100)

Epoch 1/100
43/43 [==============================] - 200s 5s/step - loss: 0.9056 - accuracy: 0.4627 - val_loss: 0.7249 - val_accuracy: 0.4967
Epoch 2/100
43/43 [==============================] - 202s 5s/step - loss: 0.7230 - accuracy: 0.4656 - val_loss: 0.7112 - val_accuracy: 0.4922
Epoch 3/100
43/43 [==============================] - 202s 5s/step - loss: 0.7057 - accuracy: 0.4832 - val_loss: 0.7060 - val_accuracy: 0.4822
Epoch 4/100
43/43 [==============================] - 202s 5s/step - loss: 0.6969 - accuracy: 0.4861 - val_loss: 0.7048 - val_accuracy: 0.4844
Epoch 5/100
43/43 [==============================] - 202s 5s/step - loss: 0.6931 - accuracy: 0.4963 - val_loss: 0.7064 - val_accuracy: 0.4922
Epoch 6/100
43/43 [==============================] - 202s 5s/step - loss: 0.6905 - accuracy: 0.5190 - val_loss: 0.7071 - val_accuracy: 0.4878
Epoch 7/100
43/43 [==============================] - 202s 5s/step - loss: 0.6896 - accuracy: 0.5270 - val_loss: 0.7081 - val_accuracy: 0.5144
Epoch 

KeyboardInterrupt: 

In [ ]:

from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
from keras.callbacks import History 
history = History()

In [ ]:
def plot_confusion_matrix(cm, classes, 
                          normalize=False,
                         title = 'Confusion Matrix',
                         cmap=plt.cm.Blues):
  
  plt.imshow(cm, interpolation = 'nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation = 45)
  plt.yticks(tick_marks, classes)
  
  if normalize:
    cm = cm.astype('float') / cm.sum(axis=1) [:, np.newaxis]
    print("Normalized Confusion Matrix")
  else:
    print("Confusion Matrix without normalization")
  
  print(cm)
  
  thresh = cm.max() / 2.
  
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j,i, cm[i,j],
            horizontalalignment ="center",
            color = "white" if cm[i,j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
  
  

In [ ]:
import numpy as np
def plots(ims, figsize=(12,6), rows=5, interp=False, titles=None):
  if type(ims[0]) is np.ndarray:
    ims = np.array(ims).astype(np.uint8)
    if(ims.shape[-1] != 3):
      ims = ims.transpose ((0,2,3,1))
  f = plt.figure(figsize = figsize)
  cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows +1
  for i in range(len(ims)):
    sp = f.add_subplot (rows, cols, i+1)
    sp.axis('Off')
    if titles is not None:
      sp.set_title(titles[i], fontsize =10)
    plt.imshow(ims[i], interpolation=None if interp else 'none')

In [ ]:
# test_path = r"/content/drive/MyDrive/dummy/dataset/test"
test_path = (r"/content/drive/MyDrive/dummy/dataset/test")
test_batches_fine = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, 
       height_shift_range=0.1, shear_range=0.15, zoom_range=0.1, 
       channel_shift_range=10., horizontal_flip=True).flow_from_directory(test_path, target_size=(256,256), classes =['Class0','Class1'], batch_size=50)

In [ ]:
test_imgs, test_labels = next(test_batches_fine)

In [ ]:

import matplotlib.pyplot as plt
x_labels = test_labels.argmax(axis=1)
print(x_labels)
plots(test_imgs, titles=test_labels)

In [ ]:
#test_labels = test_labels[:,0]
x_labels

In [ ]:
predictions = classifier.predict_generator(test_batches_fine, steps=1, verbose=0)

In [ ]:
y_labels = predictions.argmax(axis=1)
y_labels
#np.round(predictions[:,0])

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(x_labels, y_labels)

In [ ]:
cm_plot_labels = ['Normal','Glaucoma']

In [ ]:
from sklearn.metrics import plot_confusion_matrix
# plot_confusion_matrix(cm, cm_plot_labels,title='confusion matrix')
cm_plot_title =['Confusion Matrix']
plot_confusion_matrix(cm, cm_plot_labels, title ='Confusion Matrix')

In [ ]:
from keras.callbacks import History 
history = History()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()